金門縣政府提供教育相關數據

https://drive.google.com/drive/folders/1ZnEmLN_Gbx074-dv_8m3chn2vvhaeUSK?usp=sharing

金門縣康軒國中小使用數據

https://docs.google.com/spreadsheets/d/1LvcLpeVNIa-OfICFnakSbzQgSKBCkfFACmQRIKekgRA/edit?usp=sharing

In [107]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [108]:
import pandas as pd
# read data and put it in a dataframe
# 在 google 工作表載入 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1qpTHMQSEeyR9q2yR1Byh9TfnMioQEFNt8l7owvJFrOE/edit?usp=sharing')

In [109]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')


In [110]:
# 從 gsheets 的 All-whiteboard-device 載入 sheets
sheets = gsheets.worksheet('students').get_all_values()

# 將 sheets1 資料載入 pd 的 DataFrame 進行分析
df = pd.DataFrame(sheets[1:], columns=sheets[0])
# 取得最前面的5筆資料
df.head()

,id,json,timestamp
0,281746,"{""name"": ""王小明"", ""age"": 16, ""gender"": ""男"", ""add...",1743039328
1,281747,"{""name"": ""李大華"", ""age"": 17, ""gender"": ""男"", ""add...",1743039328
2,281748,"{""name"": ""張小美"", ""age"": 16, ""gender"": ""女"", ""add...",1743039328
3,281749,"{""name"": ""陳志明"", ""age"": 18, ""gender"": ""男"", ""add...",1743039328
4,281750,"{""name"": ""林佳慧"", ""age"": 17, ""gender"": ""女"", ""add...",1743039328


In [111]:
df['timestamp']

,timestamp
0,1743039328
1,1743039328
2,1743039328
3,1743039328
4,1743039328
5,1743039328
6,1743039328


In [112]:
alldays.days

0

In [113]:
len(df)

7

In [114]:
import json
df_expanded = pd.json_normalize(df['json'].apply(json.loads))

In [115]:
df_expanded.columns = df_expanded.columns.str.strip()  # 移除空格
print(df_expanded.columns)

Index(['name', 'age', 'gender', 'address', 'gpa'], dtype='object')


In [116]:
type(df_expanded['name'][0])

str

In [117]:
type(df_expanded['age'][0])

numpy.int64

In [118]:
type(df_expanded['gender'][0])

str

In [119]:
type(df_expanded[ 'address'][0])

str

In [120]:
type(df_expanded[ 'gpa'][0])

numpy.float64

In [121]:
print(df.columns)



Index(['id', 'json', 'timestamp'], dtype='object')


In [122]:
print(df['json'].head())


0    {"name": "王小明", "age": 16, "gender": "男", "add...
1    {"name": "李大華", "age": 17, "gender": "男", "add...
2    {"name": "張小美", "age": 16, "gender": "女", "add...
3    {"name": "陳志明", "age": 18, "gender": "男", "add...
4    {"name": "林佳慧", "age": 17, "gender": "女", "add...
Name: json, dtype: object


In [123]:
import json

# 将 'json' 列中的字符串转换为字典
df['json'] = df['json'].apply(json.loads)


In [124]:
from pandas import json_normalize

# 使用 json_normalize 展开 json 列
df_flattened = json_normalize(df['json'])

# 将展开后的数据合并到原 DataFrame 中
df_expanded = pd.concat([df, df_flattened], axis=1)

# 查看合并后的 DataFrame
print(df_expanded.head())


       id                                               json   timestamp name  \
0  281746  {'name': '王小明', 'age': 16, 'gender': '男', 'add...  1743039328  王小明   
1  281747  {'name': '李大華', 'age': 17, 'gender': '男', 'add...  1743039328  李大華   
2  281748  {'name': '張小美', 'age': 16, 'gender': '女', 'add...  1743039328  張小美   
3  281749  {'name': '陳志明', 'age': 18, 'gender': '男', 'add...  1743039328  陳志明   
4  281750  {'name': '林佳慧', 'age': 17, 'gender': '女', 'add...  1743039328  林佳慧   

   age gender address  gpa  
0   16      男     台北市  3.8  
1   17      男     新北市  3.5  
2   16      女     桃園市  3.9  
3   18      男     台中市  3.2  
4   17      女     台南市  3.7  


In [125]:
df_expanded['name'] = df_expanded['json'].apply(lambda x: x['name'])
df_expanded['age'] = df_expanded['json'].apply(lambda x: x['age'])
df_expanded['gender'] = df_expanded['json'].apply(lambda x: x['gender'])
df_expanded['address'] = df_expanded['json'].apply(lambda x: x['address'])
df_expanded['gpa'] = df_expanded['json'].apply(lambda x: x.get('gpa', None))  # 如果不存在 gpa，返回 None

# 删除原始的 json 列
df_expanded.drop(columns=['json'], inplace=True)

# 查看结果
df_expanded.head()


,id,timestamp,name,age,gender,address,gpa
0,281746,1743039328,王小明,16,男,台北市,3.8
1,281747,1743039328,李大華,17,男,新北市,3.5
2,281748,1743039328,張小美,16,女,桃園市,3.9
3,281749,1743039328,陳志明,18,男,台中市,3.2
4,281750,1743039328,林佳慧,17,女,台南市,3.7


In [126]:
allStudents = len(df_expanded['age'].unique())

In [127]:
allStudents

3

In [128]:
students_17 = df_expanded[df_expanded['age'] == 17]


In [129]:
students_17

,id,timestamp,name,age,gender,address,gpa
1,281747,1743039328,李大華,17,男,新北市,3.5
4,281750,1743039328,林佳慧,17,女,台南市,3.7


In [130]:
import pandas as pd
import plotly.express as px
from sklearn.cluster import KMeans

In [131]:
df_expanded

,id,timestamp,name,age,gender,address,gpa
0,281746,1743039328,王小明,16,男,台北市,3.8
1,281747,1743039328,李大華,17,男,新北市,3.5
2,281748,1743039328,張小美,16,女,桃園市,3.9
3,281749,1743039328,陳志明,18,男,台中市,3.2
4,281750,1743039328,林佳慧,17,女,台南市,3.7
5,282943,1743039328,黃麗芬,18,女,高雄市,3.6
6,282944,1743039328,劉建國,16,男,新竹市,3.4


In [132]:
# 进行 K-means 聚类
X = df_expanded[['age', 'gpa']]
kmeans = KMeans(n_clusters=3)
df_expanded['cluster'] = kmeans.fit_predict(X)

# 检查 df_expanded 是否包含 cluster 列
print(df_expanded.columns)

Index(['id', 'timestamp', 'name', 'age', 'gender', 'address', 'gpa',
       'cluster'],
      dtype='object')


In [133]:
# 使用 Plotly 绘制散点图
fig = px.scatter(df_expanded, x='age', y='gpa', color='cluster',
                 hover_data=['name', 'gender', 'address'],
                 title="Students' GPA vs Age by Cluster")

# 展示图表
fig.show()

In [134]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 1. 标准化数据
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_expanded[['age', 'gpa']])

# 2. 进行 PCA 降维到 2 个主成分
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_scaled)

# 将主成分加入原始数据
df_expanded['PCA1'] = pca_components[:, 0]
df_expanded['PCA2'] = pca_components[:, 1]

# 3. 使用 Plotly 绘制 PCA 结果
fig = px.scatter(df_expanded, x='PCA1', y='PCA2',
                 color='age',
                 title="PCA of Age and GPA",
                 labels={'PCA1': 'Principal Component 1', 'PCA2': 'Principal Component 2'})
fig.show()

In [135]:
# 1. 将 'gender' 列转换为数值，假设 '男' -> 0, '女' -> 1
df_expanded['gender'] = df_expanded['gender'].map({'男': 0, '女': 1})

# 2. 标准化数据
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_expanded[['age', 'gpa', 'gender']])

# 3. 进行 PCA 降维到 2 个主成分
pca = PCA(n_components=2)
pca_components = pca.fit_transform(df_scaled)

# 将主成分加入原始数据
df_expanded['PCA1'] = pca_components[:, 0]
df_expanded['PCA2'] = pca_components[:, 1]

# 4. 使用 Plotly 绘制 PCA 结果
fig = px.scatter(df_expanded, x='PCA1', y='PCA2',
                 color='age',
                 title="PCA of Age, GPA, and Gender",
                 labels={'PCA1': 'Principal Component 1', 'PCA2': 'Principal Component 2'})
fig.show()


Grouping and Counting Behaviors per Theme